# Analyse failed trades

Analyse failed trades over multiple trade executors.

 - Show failed and repaired and fix trades
  - Analyse slippage parameters and other causes of failure



In [1]:
from pathlib import Path
from tradeexecutor.state.state import State

state = State.read_json_file(Path("../enzyme-polygon-matic-usdc.json"))

/Users/moo/Library/Caches/pypoetry/virtualenvs/trade-executor-0IMGzMxl-py3.10/lib/python3.10/site-packages/dataclasses_json/core.py:187: RuntimeWarning: `NoneType` object value of non-optional type initial_cash detected when decoding TradeSummary.
  warnings.warn(


# Vault status

In [2]:
treasury = state.sync.treasury
deployment = state.sync.deployment

print(f"Vault deployment: {deployment}")
print(f"Treasury sync status: {treasury}")

Vault deployment: <Deployment chain:polygon address:0x8b6a97D4e4F16f79D8A59B36DD57F33ac3B686bD name:DEGEN II token:DEGE2>
Treasury sync status: <Treasury updated:2023-07-28 04:01:53 cycle:2023-07-28 04:00:00 block scanned:45,605,394 refs:7>


In [3]:

from tradeexecutor.state.trade import TradeExecution

all_trades = list(state.portfolio.get_all_trades())

t: TradeExecution
failed_trades = [t for t in all_trades if t.is_failed() or t.is_repaired() or t.is_repair_trade()]

print(f"Total {len(failed_trades)} failed and repaired trades")

Total 11 failed and repaired trades


In [4]:
import pandas as pd
from tradeexecutor.analysis.slippage import display_slippage


pd.set_option('display.max_colwidth', None)

df = display_slippage(failed_trades)
display(df)

,Flags,Position,Time,Trade,Lag,Slippage,amountIn,amountOut,Enzyme amountOut,Assumed price,Uniswap price,Block decided,Block executed,Failure reason,Tx
5,REP,#3,2023-07-03 07:45:56,USDC->WMATIC,11.0,100.0,1.475939,2.269622546029597952,2.247151013201358641,0.650237,0.6503015237410108133085080401,nan,"44,502,111",Too little received                                        ,0xbfd1d3b1e1ec6c5ce4bf37aabaf07d6ae8b4eca945e5438db0ce93a6a9534f0b
8,FIX,#3,2023-07-03 07:45:56,WMATIC->USDC,,,,,,0.650237,,nan,nan,,
4,REP,#2,2023-07-03 07:45:56,WMATIC->USDC,,100.0,4.603424392792025237,2.886265,2.857688,0.627045,0.6269821666929670082936444804,nan,"44,480,610",Too little received                                        ,0xc3f674c79fef3fedea13b24e26d811bbb2d49edffd8c1b6c332c81032ef58e52
7,FIX,#2,2023-07-03 07:45:56,USDC->WMATIC,,,,,,0.627045,,nan,nan,,
11,REP,#5,2023-07-06 11:46:58,USDC->WMATIC,6.0,100.0,6.348186,9.180554910137920512,9.088749270149046272,0.691413,0.6914817309125631585197820098,nan,"44,745,745",Too little received                                        ,0xe979c86b847b25757ed25e6a975eb473e7f126a1fadb9f7823f24524622d4e32
12,FIX,#5,2023-07-06 11:46:58,WMATIC->USDC,,,,,,0.691413,,nan,nan,,
16,REP,#7,2023-07-13 23:49:59,WMATIC->USDC,,100.0,5.849075641784785089,4.90357,4.854255,0.838385,0.8383495615905090608647988530,nan,"45,037,543",Too little received                                        ,0xc1a47bdedc8acb78de5eb803af313d3867a3adfa313b55e4d7e07a1fdf3365f0
17,FIX,#7,2023-07-13 23:49:59,USDC->WMATIC,,,,,,0.838385,,nan,nan,,
18,REP,#7,2023-07-14 09:41:53,WMATIC->USDC,,100.0,5.849075641784785089,4.951599,4.902083,0.846646,0.8465609445408147686826286023,nan,"45,047,922",Too little received                                        ,0xcdd25e39fa7277f3eb3402b8a8ba2470bff28b2eb2097cb63abac644261fb4a5
19,FIX,#7,2023-07-14 09:41:53,USDC->WMATIC,,,,,,0.846646,,nan,nan,,
